In [ ]:

!pip install rich jedi 'lightning'
!pip install transformers tensorboardX thulac pyyaml


In [13]:

import sys
sys.path.append('.')

In [14]:
MODEL='20230419_train_bert_singledata'

In [15]:

from utils import HParam
hp = HParam('config/train_bert.yaml')

In [16]:

from kg_generator import *
from training.util import setup_data_from_args
kg_model = kg_model_entities_generator(hp.kg.pretrained_file,hp.kg.num_labels)
#data
data, tokenizer= setup_data_from_args(hp)

kg_litmodel = KG_BaseLitModel

if hp.kg.load_checkpoint != 'None':
    kg_litmodel = kg_litmodel.load_from_checkpoint(hp.litmodel.load_checkpoint, args=hp.kg, model=kg_model)
else:
    kg_litmodel = kg_litmodel(args=hp.kg,tokenizer=tokenizer, model=kg_model)

Loading KG BERT model


Some weights of the model checkpoint at files/kg_bert_final_model were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at fil

In [17]:

from pathlib import Path
from pytorch_lightning.utilities.rank_zero import rank_zero_info, rank_zero_only
@rank_zero_only
def _ensure_logging_dir(experiment_dir):
    """Create the logging directory via the rank-zero process, if necessary."""
    Path(experiment_dir).mkdir(parents=True, exist_ok=True)

In [18]:
import pytorch_lightning as pl

# Call baks
log_dir = Path("training_logs") / str(MODEL)
_ensure_logging_dir(log_dir)
logger = pl.loggers.TensorBoardLogger(log_dir)
experiment_dir = logger.log_dir

goldstar_metric = "validation/cer" if hp.kg.loss in ("transformer",) else "validation/loss"
filename_format = str(MODEL) + "epoch={epoch:04d}-validation.loss={validation/loss:.3f}"
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=2,
    filename=filename_format,
    monitor=goldstar_metric,
    mode="min",
    auto_insert_metric_name=False,
    dirpath=experiment_dir,
    every_n_epochs=hp.trainer.chkpt_every_n_epochs,
)

summary_callback = pl.callbacks.ModelSummary(max_depth=2)

callbacks = [summary_callback, checkpoint_callback]

In [9]:

trainer = pl.Trainer(devices=hp.trainer.devices,accelerator=hp.trainer.accelerator,
                    max_epochs=hp.trainer.max_epochs,
                    logger = logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer = pl.Trainer(accelerator='cpu',
                    max_epochs=hp.trainer.max_epochs,
                    logger = logger)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(


rocm set

In [16]:

import torch
torch.set_float32_matmul_precision('high')
import os
os.environ["HSA_OVERRIDE_GFX_VERSION"]="10.3.0"

In [20]:


print(data.setup)
trainer.fit(kg_litmodel, datamodule=data)


  | Name    | Type                       | Params
-------------------------------------------------------
0 | model   | BertForTokenClassification | 101 M 
1 | loss_fn | CrossEntropyLoss           | 0     
-------------------------------------------------------
101 M     Trainable params
0         Non-trainable params
101 M     Total params
406.721   Total estimated model params size (MB)


<bound method KGYULIAO.setup of <training.data_loader.test_yuliao.KGYULIAO object at 0x7fd985b71a90>>
test data example [{'text': '红方载具击伤蓝方直升机，红方步兵多名由D6绕至D7', 'input_ids': tensor([[  101, 21128, 21129, 21130,  8024, 21131, 21132,  4507,   100,  5312,
          5635,   100,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'label': tensor([1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1])}]
Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: index out of range in self